In [ ]:
#if you need to install, install these libraries.
pip install -q langchain langchain_community langchain_groq pypdf2 faiss-cpu #langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [2]:
from PyPDF2 import PdfReader
import os
from dotenv import load_dotenv

from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain

In [3]:
load_dotenv()

True

In [4]:
groq_api_key = os.getenv('GROQ_API_KEY')

In [7]:
pdf_path = './KnowledgeBase.pdf'

In [ ]:
def get_pdf_text(pdf_path):
    pdf_reader = PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    print("\n1.pdf extracted\n")
    return text

raw_text= get_pdf_text(pdf_path)


1.pdf extracted

B. Diseases  
1. Citrus Black Spot  
Citrus Black Spot is a fungal disease caused by Phyllosticta citricarpa  (formerly Guignardia 
citricarpa ), which affects citrus plants like oranges, lemons, and grapefruits.  Citrus Black Spot 
thrives in warm, humid environments, making it a significant threat in tropical and subtropical 
citrus -growing regions.  
 
Symptoms:  
Affected fruits show reddish and dark depressed  spots with brown margins. These lesions spread 
and coalesce, especially on fallen and harvested fruit. Severely affected fruit ripen prematurely 
and drops. In some cases, infected fruit that are asymptomatic at harvest may develop the 
symptoms in transport or storage. When severe symptoms develop before ma turity, fruit often drop 
resulting in significant yield loss . 
[https://www.hortifresh.org/wp -content/uploads/CitrusProductionManual_2022_online.pdf ] 
Causes:  
Citrus Black Spot is a fungal disease caused by Phyllosticta citricarpa  (formerly Gui

In [8]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size = 1000,
        chunk_overlap = 200,
        length_function = len
    )
    chunks = text_splitter.split_text(text)
    print("\n2.text splitted\n")
    return chunks

text_chunks = get_text_chunks(raw_text)


2.text splitted



In [9]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(texts = text_chunks, embedding=embeddings)
    print("\n3.vectorstored\n")
    return vectorstore

vectorstore = get_vectorstore(text_chunks)

<ipython-input-9-5b60f70bf6e8>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


3.vectorstored



In [10]:
def get_conversation_chain(vectorstore):
    llm = ChatGroq(api_key = groq_api_key, model_name= "llama-3.3-70b-versatile")

    memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm = llm,
        retriever = vectorstore.as_retriever(),
        memory = memory
    )
    print("\n4.chain created\n")
    return conversation_chain

conversation= get_conversation_chain(vectorstore)


4.chain created



<ipython-input-10-5f993596c95f>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)


In [11]:
def ask_question(conversation_chain, query):

    response = conversation_chain( {"question": query} )

    print("\n5.resonse generated\n")
    return response["answer"]

query = input("Ask a question: ")
response = ask_question(conversation, query)

Ask a question: what is canker


<ipython-input-11-6feef1868fb7>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation_chain( {"question": query} )



5.resonse generated



In [12]:
def print_response(response, query):
    print("\n6.display\n")

    print("User: ", query)
    print("\n")
    print("Assistant: ", response)

print_response(response, query)


6.display

User:  what is canker


Assistant:  Canker is a bacterial disease that affects citrus plants. It appears as brownish-yellow spots on the leaves, stems, and fruits. The disease starts spreading before the rainy season and spreads rapidly during it. It can be transmitted by leaf-miner insects, which carry the bacteria into the plant through wounds or damage caused by insects or other factors. The symptoms of canker include small, pimple-like spots that are about 1 mm in diameter and yellow in color, which later become brown and corky with sunken centers and raised edges, often surrounded by a yellow ring.
